In [1]:
from read_json import *
from spectrum_integrate import *
# from plots import *
# from source_filter import *
# from variability_estimator import *

In [2]:
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.table import Table

In [3]:
## import

path_4lac_dr3_catalog = '../4LAC_catalog_generator_v3/resulting_catalogs/agn_pop_4lac_dr3.ecsv'
table_agn_pop_4lac_dr3 = Table.read(path_4lac_dr3_catalog, format='ascii.ecsv')
df_agn_pop_4lac_dr3 = table_agn_pop_4lac_dr3.to_pandas()

df_agn_pop_4lac_dr3.head()

,Source_Name,RAJ2000,DEJ2000,Redshift,SpectrumType,Pivot_Energy,PL_Flux_Density,PL_Index,LP_Flux_Density,LP_Index,LP_beta,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Flux1000,Unc_Flux1000
0,J0001.2+4741,0.3126,47.685902,-inf,PowerLaw,2420.575195,2.156031e-14,2.271696,2.196550e-14,2.254081,0.012156,bcu,ISP,25.313953,0.675882,0.312850,1.259796e-10,2.877197e-11
1,J0001.2-0747,0.3151,-7.797100,-inf,PowerLaw,1721.603882,2.656894e-13,2.116692,2.842831e-13,2.078927,0.051182,bll,LSP,46.780693,0.406565,0.111870,7.471219e-10,5.041939e-11
2,J0001.4-0010,0.3717,-0.169900,0.461516,PowerLaw,4307.082520,6.067356e-15,1.939160,6.488817e-15,1.661223,0.132438,bll,LSP,9.272764,0.000000,10.000000,1.082246e-10,2.914440e-11
3,J0001.5+2113,0.3815,21.218300,1.106000,LogParabola,329.773743,4.683598e-11,2.654060,5.371562e-11,2.514159,0.159319,fsrq,ISP,1910.935791,0.996138,0.213478,1.347354e-09,6.172809e-11
4,J0001.6-4156,0.4165,-41.942501,-inf,PowerLaw,4047.030029,1.881541e-14,1.775175,2.131597e-14,1.693865,0.072754,bcu,HSP,26.393343,0.490977,0.175782,2.821817e-10,3.090882e-11


In [4]:
## saving our changes in a separate dataframe

df_4lac_luana_dr3 = df_agn_pop_4lac_dr3.copy()

In [5]:
df_4lac_luana_dr3 = df_4lac_luana_dr3.drop(df_4lac_luana_dr3.columns[5:11], axis=1)
df_4lac_luana_dr3 = df_4lac_luana_dr3.drop(columns=['RAJ2000', 'DEJ2000', 'Redshift', 'Flux1000', 'Unc_Flux1000'])

In [6]:
## adding spectrum flux calculations

spectrum_flux_values = []

for index, row in df_4lac_luana_dr3.iterrows():
    source_name = row['Source_Name']
    
    # Integrate from 0.1-100 GeV
    spectrum_flux, diff_flux, spec_type = integrate_spectrum_flux(source_name, df_agn_pop_4lac_dr3, 0.1, 100)
    spectrum_flux_values.append(spectrum_flux)

df_4lac_luana_dr3['Spectrum_Flux_Estimated'] = spectrum_flux_values

/Users/luanareis/Documents/GitHub/CTA_AGN_Pop_Variability/4LAC_variability_analysis/spectrum_integrate.py:71: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  flux_from_spectrum = integrate.quad(lambda x: K * ((x/E_0)**(- alpha - beta * np.log(x/E_0))),


In [7]:
source_names = df_4lac_luana_dr3['Source_Name'].tolist()

len(source_names)

3814

In [8]:
## getting a list of the downloaded sources

folder_path = '../4LAC_lightcurve_downloader_v3/resulting_catalogs/input_lightcurve_downloads_v3/fixed_indexed_lightcurves/3days_ts1_fixedindex_lightcurves'

## glob to get all .json files in the folder
json_files = glob.glob(os.path.join(folder_path, '*.json'))
json_file_names = [os.path.basename(file)[5:17] for file in json_files]

len(json_file_names)

1429

In [9]:
## creating a df for the downloaded sources

df_filtered_4lac_luana_dr3 = df_4lac_luana_dr3[df_4lac_luana_dr3['Source_Name'].isin(json_file_names)].copy()
df_filtered_4lac_luana_dr3.reset_index(drop=True, inplace=True)

print(len(df_filtered_4lac_luana_dr3))

1429


In [10]:
df_filtered_4lac_luana_dr3

,Source_Name,SpectrumType,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Spectrum_Flux_Estimated
0,J0001.2-0747,PowerLaw,bll,LSP,46.780693,0.406565,0.111870,9.825299e-09
1,J0001.5+2113,LogParabola,fsrq,ISP,1910.935791,0.996138,0.213478,6.391699e-08
2,J0003.3-1928,LogParabola,bcu,LSP,49.879936,0.591879,0.159972,3.715681e-09
3,J0004.3+4614,LogParabola,fsrq,LSP,139.036758,1.016224,0.234237,1.086787e-08
4,J0004.4-4737,PowerLaw,fsrq,LSP,153.286484,0.823498,0.188146,1.070792e-08
...,...,...,...,...,...,...,...,...
1424,J2358.0-4601,PowerLaw,bcu,LSP,30.538687,0.477398,0.157409,5.184782e-09
1425,J2358.3-1021,LogParabola,fsrq,LSP,195.553940,0.668076,0.150551,2.164930e-08
1426,J2358.3+3830,PowerLaw,bll,NaN,42.173111,0.567347,0.164919,4.248420e-09
1427,J2359.0+3922,PowerLaw,fsrq,LSP,56.146664,0.779921,0.203910,7.005793e-09


In [11]:
json_source_names = df_filtered_4lac_luana_dr3['Source_Name'].tolist()

len(json_source_names)

1429

In [12]:
bin_3days = []
bin_weekly = []
bin_monthly = []
        
for i in json_source_names:
    for j in ['3-days', 'weekly', 'monthly']:
    
        # source_data = Read_json_file(file_name, binning, index)
        source_data = Read_json_file(f'4FGL+{i}.json', j, 'fixed')
        source_df = source_data.dataframe
        source_dict = source_data.dictionary

        ## check if the number of flux upper limits is less than or equal to flux values
        if len(source_dict['flux_upper_limits']) <= len(source_dict['flux']):
            if j == '3-days':
                bin_3days.append(i)
            elif j == 'weekly':
                bin_weekly.append(i)
            elif j == 'monthly':
                bin_monthly.append(i)

In [13]:
# Print or work with the lists
print('3-days bin:', len(bin_3days))
print('Weekly bin:', len(bin_weekly))
print('Monthly bin:', len(bin_monthly))

3-days bin: 173
Weekly bin: 339
Monthly bin: 803


In [14]:
# ## iterate over binning and index options
# for binning in ['3-days', 'weekly', 'monthly']:
#     for index in ['fixed', 'free']:
#         key = f'lc_{binning.replace("-", "")}_{index}_files'
#         lightcurve_files[key] = generate_list_sources(binning, index)

#         filtered_file_list = []
#         for file_name in lightcurve_files[key]:
            

#             len_flux_upper_limits = len(source_dict['flux_upper_limits'])
#             len_flux_measur = len(source_dict['flux'])

#             if len_flux_upper_limits <= len_flux_measur:
#                 filtered_file_list.append(file_name)

#             UL_filtered_files[key] = filtered_file_list


In [15]:
## filter data

## remove if n° Upper Limits > n° flux measurements


## make the point an UL if TS < 10

## remove the flux points in which flux_error = 0, fit_convergence !=0

## exposure (flux / flux_error²) < 1e7 cm²s

In [16]:
## estimate parameters


In [17]:
df_filtered_4lac_luana_dr3.head()

,Source_Name,SpectrumType,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Spectrum_Flux_Estimated
0,J0001.2-0747,PowerLaw,bll,LSP,46.780693,0.406565,0.111870,9.825299e-09
1,J0001.5+2113,LogParabola,fsrq,ISP,1910.935791,0.996138,0.213478,6.391699e-08
2,J0003.3-1928,LogParabola,bcu,LSP,49.879936,0.591879,0.159972,3.715681e-09
3,J0004.3+4614,LogParabola,fsrq,LSP,139.036758,1.016224,0.234237,1.086787e-08
4,J0004.4-4737,PowerLaw,fsrq,LSP,153.286484,0.823498,0.188146,1.070792e-08


In [18]:
df_4lac_luana_dr3.head()

,Source_Name,SpectrumType,CLASS,SED_class,Variability_Index,Frac_Variability,Unc_Frac_Variability,Spectrum_Flux_Estimated
0,J0001.2+4741,PowerLaw,bcu,ISP,25.313953,0.675882,0.312850,1.993321e-09
1,J0001.2-0747,PowerLaw,bll,LSP,46.780693,0.406565,0.111870,9.825299e-09
2,J0001.4-0010,PowerLaw,bll,LSP,9.272764,0.000000,10.000000,8.602307e-10
3,J0001.5+2113,LogParabola,fsrq,ISP,1910.935791,0.996138,0.213478,6.391699e-08
4,J0001.6-4156,PowerLaw,bcu,HSP,26.393343,0.490977,0.175782,1.599132e-09


In [19]:
## plots and table


In [20]:
table_4lac_luana_dr3 = Table.from_pandas(df_4lac_luana_dr3)
table_filtered_4lac_luana_dr3 = Table.from_pandas(df_filtered_4lac_luana_dr3) 

table_4lac_luana_dr3.write('./resulting_catalogs/df_4lac_luana_dr3.fits', format='fits', overwrite=True)
table_4lac_luana_dr3.write('./resulting_catalogs/df_4lac_luana_dr3.ecsv', format='ascii.ecsv', overwrite=True)
table_filtered_4lac_luana_dr3.write('./resulting_catalogs/df_filtered_4lac_luana_dr3.fits', format='fits', overwrite=True)
table_filtered_4lac_luana_dr3.write('./resulting_catalogs/df_filtered_4lac_luana_dr3.ecsv', format='ascii.ecsv', overwrite=True)